### Домашнее задание №7.<br />
Вес = 1.5<br />
Максимальный балл = 150<br />
Срок сдачи - 21.05.2020<br />

<div style="text-align: justify">
В этом задании предлагается выбрать задачу и решить ее с применением любых из доступных методов машинного обучения, которые были освоены в рамках курса.
</div>

<div style="text-align: justify">
Приходящая солнечная радиация - единственный (условно) стабильный и (условно) восполняемый источник энергии на планете Земля. Кроме того, солнечная радиация - практически единственный источник энергии, питающий всю динамику атмосферы и океана. Радиационный блок в моделях атмосферы - важнейшая часть. Для оценки приходящей солнечной радиации в моделировании геофизических систем в настоящий момент чаще всего применяются т.н.. параметризации - упрощенные схемы, аппроксимирующие процессы рассеяния, поглощения, переотражения, переизлучения солнечной радиации в атмосфере. Важнейшей переменной во всех параметризациях приходящей солнечной радиации является общий балл облачности.
</div>
<div style="text-align: justify">
Для построения аккуратной параметризации потоков приходящей на поверхность коротковолновой и длинноволновой радиации нужны долговременные ряды соответствующих показаний радиометрических приборов и оценки балла общей облачности.
</div>
<div style="text-align: justify">
В Лаборатории взаимодействия океана и атмосферы и мониторинга климатических изменений ведется работа по автоматизации наблюдений за облачностью. Для этого была разработана и апробирована установка, проводящая оптическую съемку облачности с поверхности. Результатом съемки являются снимки следующего рода:
</div>

<table>
    <tr>
        <td>
            <a><img src="img/img-2016-10-22T10-18-13devID1.jpg"></a>
        </td>
        <td>
            <a><img src="img/img-2016-10-22T10-18-13devID2.jpg"></a>
        </td>
    </tr>
</table>

Одновременно со съемкой ведутся визуальные наблюдения. В результате таких наблюдений фиксируются:
- балл общей облачности, оцененный наблюдателем (Total Cloud Cover, TCC)
- баллы облачности по трем ярусам (нижний, средний, верхний), оцененные наблюдателем
- наблюдаемые типы облачности
- наблюдаемое состояние диска солнца (4 класса от "пасмурно" до "Солнце в квадрате", Sun Disk State, SDS)

Одновременно со съемкой также непрерывно в автоматическом режиме производится запись показаний радиометрического оборудования: потоки приходящей коротковолновой и длинноволновой радиации.

<div style="text-align: justify">
В этом задании предлагается решить задачу <b>аппроксимации балла общей облачности (TCC) или состояния диска Солнца (SDS)</b> по данным, описывающим снимки. Выбор типа задачи (регрессия, классификация), метрик качества, вида модели, способа предварительной обработки исходных данных, - все это остается за исследователем.
</div>

### описание исходных данных

Скачать файл с исходными данным по задаче можно по [ссылке](https://www.dropbox.com/s/9joo40islnxzn14/data.csv.zip?dl=0) (zip-файл, содержащий csv-файл `data.csv`)

<div style="text-align: justify">
<b>Исходные данные</b> - статистические характеристики изображений, рассчитанные с учетом маски, исключающей из рассмотрения черные углы изображения и конструкции парохода. Такие характеристики рассчитаны для каналов красного (R), зеленого (G), и синего (B) цветов, а также для яркости (Y) и насыщенности цвета (Sat). Для каждого из таких каналов по изображению эмпирически оценены следующие статистики:
</div>

- перцентили от 5-го до 100-го включительно (совпадает с максимальным значением) - обозначены именами переменных с постфиксом `_perc_xx` (`xx` - уровень перцентиля, например, `05` или `78`);
- минимальное значение - обозначено именем переменной с постфиксом `_min`;
- максимальное значение - обозначено именем переменной с постфиксом `_max`;
- среднее значение - обозначено именем переменной с постфиксом `_mean`;
- дисперсия $\sigma^2$ - обозначено именем переменной с постфиксом `_variance`;
- среднеквадратическое отклонение $\sigma$ - обозначено именем переменной с постфиксом `\_std`;
- третий центральный момент $\mu_3$ (коэффициент асимметрии, skewness) - обозначено именем переменной с постфиксом `\_skewness`;
- четвертый центральный момент $\mu_4$ (коэффициент эксцесса, kurtosis) - обозначено именем переменной с постфиксом `\_kurtosis`;
- среднее квадратическое - обозначено именем переменной с постфиксом `_rms`;

Таким образом, каждое изображение в исходных данных описывается 520 переменными, агрегирующими характеристики пяти цветовых характеристик (`R`,`G`,`B`,`Y`,`Sat`).

Дополнительно для каждого изображения приводятся следующие данные:

- поле `dt`: дата и время съемки (UTC) в формате `YYYY-MM-DD HH:mm:SS`;
- поле `dt_obs`: дата и время (UTC) наблюдения эксперта, поставленного в соответствие этому изображению в таком же формате;
- поле `date`: дата съемки (UTC) в формате `YYYY-MM-DD` (дублирует дату в поле `dt`).

<div style="text-align: justify">
<b>Целевые переменные</b> - балл общей облачности и состояние диска Солнца по показаниям наблюдателя:
</div>

- поле `TCC` - балл общей облачности, дискретные целые значения от 0 до 8;
- поле `SDS` - состояние диска Солнца, закодированное целыми числами от -1 до 3 по следующему правилу: (-1) - "Пасмурно"; (0) - "Солнце в нулевой степени"; (1) - "Солнце в первой степени"; "2" - "Солнце в квадрате".

На выбор исследователя также остается целевая переменная: `TCC` или `SDS`.

Следует учитывать, что существует базовое решение (т.н. "baseline"): современные алгоритмы, не применяющие методы машинного обучения, способны оценивать TCC с ошибкой не более 1 балла в 62% случаев.

<hr>

Для считывания данных можно использовать библиотеку `pandas`. Ниже приведены фрагменты кода, позволяющие считать данные

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data.csv', sep=';', parse_dates=['dt', 'dt_obs', 'date'])

In [3]:
df.head()

,Sat_perc_5,Sat_perc_6,Sat_perc_7,Sat_perc_8,Sat_perc_9,Sat_perc_10,Sat_perc_11,Sat_perc_12,Sat_perc_13,Sat_perc_14,...,B_variance,B_stdev,B_skewness,B_kurtosis,B_rms,TCC,SDS,dt,dt_obs,date
0,0.701671,0.708021,0.713109,0.717674,0.721794,0.725118,0.728264,0.731046,0.734055,0.736685,...,984.545574,31.377469,1.641334,3.368208,139.108778,0.0,2,2015-06-13 03:55:20,2015-06-13 04:00:00,2015-06-13
1,0.718301,0.722983,0.727109,0.730842,0.734771,0.738330,0.742218,0.745625,0.748853,0.752498,...,1033.562130,32.149061,1.118913,3.614911,139.010282,0.0,2,2015-06-13 03:55:20,2015-06-13 04:00:00,2015-06-13
2,0.698364,0.704743,0.709573,0.714295,0.718634,0.722025,0.725292,0.728381,0.731404,0.734355,...,995.109758,31.545360,1.631717,3.291467,139.732056,0.0,2,2015-06-13 03:55:50,2015-06-13 04:00:00,2015-06-13
3,0.715817,0.720252,0.724623,0.729077,0.732906,0.736904,0.740756,0.744403,0.748075,0.751174,...,1045.109219,32.328149,1.116329,3.570819,139.537405,0.0,2,2015-06-13 03:55:50,2015-06-13 04:00:00,2015-06-13
4,0.714426,0.719295,0.723994,0.727931,0.732238,0.735951,0.739825,0.743432,0.746728,0.749933,...,1034.084552,32.157185,1.135922,3.665238,138.296099,0.0,2,2015-06-13 03:56:20,2015-06-13 04:00:00,2015-06-13


<p> </p>

<p> </p>

<p> </p>

<hr />

В решении ожидаются следующие разделы кода с описанием происходящего:

| # | пункт решения | макс. балл |
|---|-------------|-----------|
| 1 | формулировка задачи с четким указанием целевой переменной, метрик качества, типа решаемой задачи | 10 |
| 2 | считывание данных | 5 |
| 3 | разведочный анализ и визуализация исходных данных | 15 |
| 4 | описание и обоснование предварительной обработки исходных данных | 5 |
| 5 | выбор модели (или моделей) для решения задачи и обоснование этого выбора | 5 |
| 6 | разбиение исходных данных на подвыборки: тренировочная, валидационная (при необходимости), тестовая (при необходимости); обоснование выбранного разбиения; | 15 |
| 7 | настройка модели/моделей, оптимизация/подбор гиперпараметров, оценка качества в процессе обучения | 25 |
| 8 | оценка качества финальной отобранной и обученной модели | 15 |
| 9 | оценка неопределенности решения | 25 |
| 10| визуализация результатов | 10 |
| 11| выводы на основании результатов | 20 |
|---|-------------|-----------|
| * | Качество на отложенной выборке (недоступна на этапе выполнения ДЗ) - опционально | +/- 50 |

В случае, если обсуждение ДЗ №7 выносится на зачет, качество модели проверяется на отложенной выборке (недоступна для исследователя на этапе выполнения ДЗ). Для этого следует иметь сохраненную модель, для чего можно воспользоваться средствами сохранения объектов python. Более подробно с такой возможностью можно ознакомиться [здесь](https://scikit-learn.org/stable/modules/model_persistence.html).

При проверке качества общий балл может быть как дополнен, так и снижен в зависимости от того, насколько сильно результирующее качество выпадает из интервала оценки неопределенности (см. п.9 ожидаемых стадий решения).

Для исследователей, для которых доступен автозачет и кто согласен с оценкой на автозачет, - эта опция неприменима.

#### Примечания:

- следует обратить внимание, что для каждого снимка заполнен признак даты и времени съемки. Это сделано не просто так. Следует внимательно отнестись к разбиению данных на тренировочную, валидационную и тестовую части;